## Importerer pakker

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Leser inn data

In [71]:
demographic_df = pd.read_csv('../raw_data/demographic.csv')
hospital_df = pd.read_csv('../raw_data/hospital.csv')
physiological_df = pd.read_table('../raw_data/physiological.txt')
severity_json = pd.read_json('../raw_data/severity.json')

demographic_df_sample = pd.read_csv('../sample_data/demographic.csv')
hospital_df_sample = pd.read_csv('../sample_data/hospital.csv')
physiological_df_sample = pd.read_table('../sample_data/physiological.txt')
severity_json_sample = pd.read_json('../sample_data/severity.json')
pd.set_option('display.max_columns', None)


## Omgjør Severity_json til DataFrame

In [72]:
# Eksploderer alle kolonner fra indeks 2 til siste kolonne for hver oppføring.
# Dette vil utvide DataFrame-en slik at hver verdi i disse kolonnene blir en egen rad
severity_df = severity_json.iloc[:, 0:].explode(list(severity_json.columns[2:]))

# Tilbakestiller indeksen etter eksplosjonen slik at radene får sammenhengende nummererin
severity_df.reset_index(drop=True, inplace=True)

# Sorterer DataFrame-en etter 'pasient_id' for å sikre at radene er organisert etter pasient
severity_df = severity_df.sort_values(by=['pasient_id'], ignore_index=True)
severity_df

# Gjentar de samme operasjonene for en sample-DataFrame (severity_json_sample)
# Eksploderer alle kolonner fra indeks 2 til siste kolonne
severity_df_sample = severity_json_sample.iloc[:, 0:].explode(list(severity_json_sample.columns[2:]))

# Tilbakestiller indeksen for prøven slik at radene får sammenhengende nummerering
severity_df_sample.reset_index(drop=True, inplace=True)

# Sorterer sample-DataFrame-en etter 'pasient_id' for konsistent organisering
severity_df_sample = severity_df_sample.sort_values(by=['pasient_id'], ignore_index=True)

severity_df_sample

sykdomskategori_id     sykdomskategori pasient_id dødsfall  \
0                   ChE              Cancer          1        0   
1                   ChE              Cancer         22        1   
2                   ChE              Cancer         25        1   
3                   BrY  COPD/CHF/Cirrhosis         26        1   
4                   A1s            ARF/MOSF         29        0   
...                 ...                 ...        ...      ...   
1360                A1s            ARF/MOSF       9049        0   
1361                ChE              Cancer       9050        1   
1362                A1s            ARF/MOSF       9078        0   
1363                ChE              Cancer       9080        1   
1364                A1s            ARF/MOSF       9081        0   

     sykdom_underkategori antall_komorbiditeter koma_score adl_pasient  \
0             Lung Cancer                     0        0.0         7.0   
1             Lung Cancer                     0        0.0        None   
2            Colon Cancer                     0        0.0        None   
3                    COPD                     1        0.0         5.0   
4       ARF/MOSF w/Sepsis                     2       37.0        None   
...                   ...                   ...        ...         ...   
1360    ARF/MOSF w/Sepsis                     0        0.0         0.0   
1361         Colon Cancer                     2        0.0         0.0   
1362    ARF/MOSF w/Sepsis                     3        0.0        None   
1363          Lung Cancer                     2        0.0        None   
1364    ARF/MOSF w/Sepsis                     1        0.0        None   

     adl_stedfortreder fysiologisk_score apache_fysiologisk_score  \
0                  7.0         33.898438                     20.0   
1                 None         26.199219                     35.0   
2                 None         18.898438                     10.0   
3                  4.0         14.199219                     16.0   
4                 None         22.398438                     21.0   
...                ...               ...                      ...   
1360               0.0         16.898438                     20.0   
1361               0.0         18.097656                     13.0   
1362               2.0         17.796875                      8.0   
1363              None              11.0                     18.0   
1364               2.0         35.296875                     70.0   

     overlevelsesestimat_2mnd overlevelsesestimat_6mnd diabetes demens  \
0                    0.262939                 0.036995        0      0   
1                    0.535889                 0.213989        0      0   
2                    0.853882                 0.674927        0      0   
3                    0.939941                 0.897949        0      0   
4                    0.823975                 0.764893        0      0   
...                       ...                      ...      ...    ...   
1360                 0.856934                 0.807983        0      0   
1361                 0.809937                 0.592896        0      0   
1362                 0.882935                 0.841919        1      0   
1363                 0.827881                 0.625977        1      0   
1364                 0.662964                 0.564941        1      0   

           kreft lege_overlevelsesestimat_2mnd lege_overlevelsesestimat_6mnd  \
0     metastatic                           0.5                          0.25   
1     metastatic                           0.5                         0.125   
2     metastatic                           0.5                           0.1   
3             no                           0.9                           0.6   
4             no                          None                          None   
...          ...                           ...                           ...   
1360          no                 

## Slår sammen DataFrames

In [73]:
# Slår sammen demografi- og sykehusdata basert på 'pasient_id' med venstre-join for å beholde alle rader fra demografi
demographic_hospital_df = demographic_df.merge(hospital_df, on='pasient_id', how='left')

# Slår sammen den resulterende DataFrame med fysiologiske data basert på 'pasient_id' med indre join
# Dette antar at alle pasienter i demographic_hospital_df har tilsvarende data i physiological_df
demographic_hospital_phisiological_df = demographic_hospital_df.merge(physiological_df, on='pasient_id')

# Slår sammen med alvorlighetsdata for å legge til alvorlighetsnivåer for hver pasient
pasient_info_df = demographic_hospital_phisiological_df.merge(severity_df, on='pasient_id')

# For prøvedata: Konverterer 'pasient_id' i severity_df_sample til int64 for å sikre kompatibilitet under sammenslåing
severity_df_sample['pasient_id'] = severity_df_sample['pasient_id'].astype('int64')

# Slår sammen demografi- og sykehusdata i prøvedata basert på 'pasient_id' med venstre-join
demographic_hospital_df_sample = demographic_df_sample.merge(hospital_df_sample, on='pasient_id', how='left')

# Slår sammen med fysiologiske data i prøvedata basert på 'pasient_id' med indre join
demographic_hospital_phisiological_df_sample = demographic_hospital_df_sample.merge(physiological_df_sample, on='pasient_id')

# Slår sammen prøvedata med alvorlighetsdata for å inkludere alvorlighetsnivåer for hver pasient
pasient_info_df_sample = demographic_hospital_phisiological_df_sample.merge(severity_df_sample, on='pasient_id')


## Dropper duplikat pasient_ider

In [74]:
pasient_info_df = pasient_info_df.drop_duplicates()
pasient_info_df_sample = pasient_info_df.drop_duplicates()

Dropper duplikate pasient_ider da disse pasientene hadde de samme verdiene.

## Fjerner pasienter med negativ oppholdslengde

In [75]:
pasient_info_df = pasient_info_df[pasient_info_df['oppholdslengde']>0]

Filtrerer ut rader der 'oppholdslengde' er 0 eller mindre, ettersom disse verdiene indikerer ugyldige eller manglende oppholdslengder.
Dette er viktig for å sikre at modellen kun trenes på pasienter med faktisk oppholdstid, og unngå at nullverdier påvirker prediksjonen negativt.

## Runder av alder til heltall, og setter negativ alder til NaN

In [76]:
pasient_info_df.loc[pasient_info_df['alder'] < 0, 'alder'] = np.nan
pasient_info_df['alder'] = pasient_info_df['alder'].round().astype('Int64')
pasient_info_df

pasient_info_df_sample.loc[pasient_info_df_sample['alder'] < 0, 'alder'] = np.nan
pasient_info_df_sample['alder'] = pasient_info_df_sample['alder'].round().astype('Int64')
pasient_info_df


pasient_id  alder   kjønn  utdanning     inntekt etnisitet  sykehusdød  \
0             2     60  female       12.0    $11-$25k     white           1   
1             3     53  female       12.0  under $11k     white           0   
2             4     42  female       11.0  under $11k     white           0   
4             6     93    male       14.0         NaN     white           1   
5             7     62    male       14.0    $25-$50k     white           0   
...         ...    ...     ...        ...         ...       ...         ...   
7735       9101     66    male        8.0         NaN     white           0   
7736       9102     55  female       11.0         NaN     white           0   
7737       9103     70    male        NaN         NaN     white           0   
7738       9104     47    male       13.0         NaN     white           1   
7739       9105     82  female        8.0    $11-$25k     white           0   

      oppholdslengde  blodtrykk  hvite_blodlegemer  hjertefrekvens  \
0                  4       43.0          17.097656           112.0   
1                 17       70.0           8.500000            88.0   
2                  3       75.0           9.099609            88.0   
4                  4      110.0          10.398438           101.0   
5                  9       78.0          11.699219           120.0   
...              ...        ...                ...             ...   
7735              23      109.0           7.399414           104.0   
7736              29       43.0                NaN             0.0   
7737               8      111.0           8.398438            83.0   
7738               7       99.0           7.599609           110.0   
7739              12       75.0           8.599609            69.0   

      respirasjonsfrekvens  kroppstemperatur  lungefunksjon  serumalbumin  \
0                     34.0          34.59375       98.00000           NaN   
1                     28.0          37.39844      231.65625           NaN   
2                     32.0          35.00000            NaN           NaN   
4                     44.0          38.39844      266.62500           NaN   
5                     28.0          37.39844      309.50000      4.799805   
...                    ...               ...            ...           ...   
7735                  22.0          35.69531      280.00000      3.699707   
7736                   8.0          38.59375      218.50000           NaN   
7737                  24.0          36.69531      180.00000           NaN   
7738                  24.0          36.39844      428.56250      1.199951   
7739                  24.0          36.19531      230.40625      4.500000   

      bilirubin  kreatinin  natrium   blod_ph  glukose  blodurea_nitrogen  \
0           NaN   5.500000    132.0  7.250000      NaN                NaN   
1      2.199707   2.000000    134.0  7.459961      NaN                NaN   
2           NaN   0.799927    139.0       NaN      NaN                NaN   
4           NaN   0.699951    140.0  7.659180      NaN                NaN   
5      0.399963   1.599854    132.0  7.479492      NaN                NaN   
...         ...        ...      ...       ...      ...                ...   
7735   0.399963   1.099854    131.0  7.459961    188.0               21.0   
7736        NaN   5.899414    135.0  7.289062    190.0               49.0   
7737   0.399963   2.699707    139.0  7.379883    189.0               60.0   
7738   0.399963   3.500000    135.0  7.469727    246.0               55.0   
7739   0.599976   1.199951    137.0  7.289062    187.0               15.0   

      urinmengde sykdomskategori_id     sykdomskategori dødsfall  \
0            NaN                BrY  COPD/CHF/Cirrhosis        1   
1            NaN                BrY  COPD/CHF/Cirrhosis        1   
2            NaN                ChE              Cancer        1   
4            NaN                DWw                Coma        1   
5            NaN         

Setter 'alder' til NaN for pasienter med negativ alder, siden negativ alder er ugyldig
Dette er viktig for å unngå at modeller og analyser blir påvirket av ugyldige aldersverdier

## Setter ugyldige verdier til NaN

In [77]:
pasient_info_df.loc[pasient_info_df['blodtrykk'] <= 0, 'blodtrykk'] = np.nan
pasient_info_df.loc[pasient_info_df['hjertefrekvens'] <= 0, 'hjertefrekvens'] = np.nan
pasient_info_df.loc[pasient_info_df['respirasjonsfrekvens'] <= 0, 'respirasjonsfrekvens'] = np.nan
pasient_info_df.loc[pasient_info_df['urinmengde'] <= 0, 'urinmengde'] = np.nan
pasient_info_df.loc[pasient_info_df['glukose'] <= 0, 'glukose'] = np.nan

pasient_info_df_sample.loc[pasient_info_df_sample['blodtrykk'] <= 0, 'blodtrykk'] = np.nan
pasient_info_df_sample.loc[pasient_info_df_sample['hjertefrekvens'] <= 0, 'hjertefrekvens'] = np.nan
pasient_info_df_sample.loc[pasient_info_df_sample['respirasjonsfrekvens'] <= 0, 'respirasjonsfrekvens'] = np.nan
pasient_info_df_sample.loc[pasient_info_df_sample['urinmengde'] <= 0, 'urinmengde'] = np.nan
pasient_info_df_sample.loc[pasient_info_df_sample['glukose'] <= 0, 'glukose'] = np.nan


Setter måleverdier til NaN for pasienter med urealistiske eller ugyldige verdier i hoved-DataFrame
Dette sikrer at verdier på 0 eller mindre, som indikerer feil eller manglende data, ikke påvirker analysen

# Fjerning av variabler

## Sykdomskategori er redundant, da den vil være lik sykdomskategori

In [78]:
pasient_info_df.drop(columns={'sykdomskategori_id'}, inplace=True)

pasient_info_df_sample.drop(columns={'sykdomskategori_id'}, inplace=True)

## Fjerner døsfall, da dette ikke er nødvendig å inkludere om en pasient har død etter sykehusoppholdet

In [79]:
pasient_info_df.drop(columns={'dødsfall'}, inplace=True)

pasient_info_df_sample.drop(columns={'dødsfall'}, inplace=True)

Fjerner kolonnen 'dødsfall' fra DataFrame-en ettersom den inneholder generell dødsinformasjon som ikke nødvendigvis er relevant for sykehusoppholdet.
'sykehusdød' beholdes fordi den spesifikt indikerer om pasienten døde under sykehusoppholdet, noe som kan være en viktig faktor i analyser og modelltrening knyttet til oppholdslengde.

## Fjerner variabler som ikke er utfylt på dag 1

In [80]:
pasient_info_df.drop(columns={'bilirubin'}, inplace=True)
pasient_info_df.drop(columns={'adl_pasient'}, inplace=True)

pasient_info_df_sample.drop(columns={'bilirubin'}, inplace=True)
pasient_info_df_sample.drop(columns={'adl_pasient'}, inplace=True)

Fjerner kolonnene 'bilirubin' og 'adl_pasient' ettersom disse ikke er registrert på dag 1
Ved å fjerne variabler som ikke er utfylt på dag 1, sikrer vi at analysen og modelltreningen kun bruker data som er tilgjengelig fra første dag av sykehusoppholdet,
noe som gjør prediksjonsmodellen mer pålitelig og anvendbar for tidlige vurderinger.

## Fjerner dnr_dag, da dnr_status allerede forteller om en pasient fikk dnr status før eller etter innleggelse

In [81]:
pasient_info_df.drop(columns={'dnr_dag'}, inplace=True)

Fjerner kolonnen 'dnr_dag' ettersom 'dnr_status' allerede gir tilstrekkelig informasjon om en pasient har fått DNR-status før eller etter innleggelse.
'dnr_dag' inneholder detaljert informasjon om hvilken dag DNR-status ble gitt, men mange av disse verdiene kan være manglende eller ufullstendige,
noe som gjør imputering vanskelig og kan føre til usikkerhet i modellene. 
Ved å bruke 'dnr_status' unngår vi utfordringer knyttet til manglende verdier og komplisert imputering, samtidig som vi beholder nødvendig informasjon om DNR-status.

# Generell opprydning av datasettet

## Rydder opp ved å fjerne pasient_id og sette oppholdslengde som første rad

In [82]:
col_to_move = pasient_info_df.pop('oppholdslengde')
pasient_info_df.insert(0, 'oppholdslengde', col_to_move)
pasient_info_df.drop(columns=['pasient_id'], inplace=True)
pasient_info_df

col_to_move = pasient_info_df_sample.pop('oppholdslengde')
pasient_info_df_sample.insert(0, 'oppholdslengde', col_to_move)
pasient_info_df_sample

oppholdslengde pasient_id  alder   kjønn  utdanning     inntekt  \
0                  4          2     60  female       12.0    $11-$25k   
1                 17          3     53  female       12.0  under $11k   
2                  3          4     42  female       11.0  under $11k   
3                -99          5     80  female        NaN         NaN   
4                  4          6     93    male       14.0         NaN   
...              ...        ...    ...     ...        ...         ...   
7735              23       9101     66    male        8.0         NaN   
7736              29       9102     55  female       11.0         NaN   
7737               8       9103     70    male        NaN         NaN   
7738               7       9104     47    male       13.0         NaN   
7739              12       9105     82  female        8.0    $11-$25k   

     etnisitet  sykehusdød  blodtrykk  hvite_blodlegemer  hjertefrekvens  \
0        white           1       43.0          17.097656           112.0   
1        white           0       70.0           8.500000            88.0   
2        white           0       75.0           9.099609            88.0   
3        white           0       59.0          13.500000           112.0   
4        white           1      110.0          10.398438           101.0   
...        ...         ...        ...                ...             ...   
7735     white           0      109.0           7.399414           104.0   
7736     white           0       43.0                NaN             NaN   
7737     white           0      111.0           8.398438            83.0   
7738     white           1       99.0           7.599609           110.0   
7739     white           0       75.0           8.599609            69.0   

      respirasjonsfrekvens  kroppstemperatur  lungefunksjon  serumalbumin  \
0                     34.0          34.59375       98.00000           NaN   
1                     28.0          37.39844      231.65625           NaN   
2                     32.0          35.00000            NaN           NaN   
3                     20.0          37.89844      173.31250           NaN   
4                     44.0          38.39844      266.62500           NaN   
...                    ...               ...            ...           ...   
7735                  22.0          35.69531      280.00000      3.699707   
7736                   8.0          38.59375      218.50000           NaN   
7737                  24.0          36.69531      180.00000           NaN   
7738                  24.0          36.39844      428.56250      1.199951   
7739                  24.0          36.19531      230.40625      4.500000   

      kreatinin  natrium   blod_ph  glukose  blodurea_nitrogen  urinmengde  \
0      5.500000    132.0  7.250000      NaN                NaN         NaN   
1      2.000000    134.0  7.459961      NaN                NaN         NaN   
2      0.799927    139.0       NaN      NaN                NaN         NaN   
3      0.799927    143.0  7.509766      NaN                NaN         NaN   
4      0.699951    140.0  7.659180      NaN                NaN         NaN   
...         ...      ...       ...      ...                ...         ...   
7735   1.099854    131.0  7.459961    188.0               21.0         NaN   
7736   5.899414    135.0  7.289062    190.0               49.0         NaN   
7737   2.699707    139.0  7.379883    189.0               60.0      3900.0   
7738   3.500000    135.0  7.469727    246.0               55.0         NaN   
7739   1.199951    137.0  7.289062    187.0               15.0         NaN   

         sykdomskategori sykdom_underkategori antall_komorbiditeter  \
0     COPD/CHF/Cirrhosis            Cirrhosis                     2   
1     COPD/CHF/Cirrhosis            Cirrhosis                     2   
2                 Cancer          Lung Cancer                     2   
3               ARF/MOSF    ARF/MOSF w/Sepsis                     1   

Flytter kolonnen 'oppholdslengde' til første posisjon i DataFrame-en for å gjøre den mer synlig,
siden 'oppholdslengde' ofte er målet (target) for prediksjon i analyser.
Fjerner kolonnen 'pasient_id' fra DataFrame-en, ettersom pasient-ID ikke bidrar til modellens prediksjonskraft
og kan heller bli en distraksjon under analysen. Det er også viktig å fjerne ID-er for å anonymisere data.

## Sjekker data

In [83]:
pasient_info_df.dtypes

oppholdslengde                     int64
alder                              Int64
kjønn                             object
utdanning                        float64
inntekt                           object
etnisitet                         object
sykehusdød                         int64
blodtrykk                        float64
hvite_blodlegemer                float64
hjertefrekvens                   float64
respirasjonsfrekvens             float64
kroppstemperatur                 float64
lungefunksjon                    float64
serumalbumin                     float64
kreatinin                        float64
natrium                          float64
blod_ph                          float64
glukose                          float64
blodurea_nitrogen                float64
urinmengde                       float64
sykdomskategori                   object
sykdom_underkategori              object
antall_komorbiditeter             object
koma_score                        object
adl_stedfortrede

## Omgjør variabeler til sin respektive type

In [84]:
columns_category = pasient_info_df.select_dtypes(include=['object']).columns.tolist()
columns_category

['kjønn',
 'inntekt',
 'etnisitet',
 'sykdomskategori',
 'sykdom_underkategori',
 'antall_komorbiditeter',
 'koma_score',
 'adl_stedfortreder',
 'fysiologisk_score',
 'apache_fysiologisk_score',
 'overlevelsesestimat_2mnd',
 'overlevelsesestimat_6mnd',
 'diabetes',
 'demens',
 'kreft',
 'lege_overlevelsesestimat_2mnd',
 'lege_overlevelsesestimat_6mnd',
 'dnr_status']

In [85]:
to_int_columns = ['antall_komorbiditeter', 'diabetes', 'demens']
to_float_columns = ['koma_score', 'fysiologisk_score', 'apache_fysiologisk_score', 'overlevelsesestimat_2mnd', 'overlevelsesestimat_6mnd', 'lege_overlevelsesestimat_2mnd', 'lege_overlevelsesestimat_6mnd', 'adl_stedfortreder']

In [86]:
pasient_info_df[to_int_columns] = pasient_info_df[to_int_columns].astype(dtype=int)
pasient_info_df[to_float_columns] = pasient_info_df[to_float_columns].astype(dtype=float)

pasient_info_df_sample[to_int_columns] = pasient_info_df_sample[to_int_columns].astype(dtype=int)
pasient_info_df_sample[to_float_columns] = pasient_info_df_sample[to_float_columns].astype(dtype=float)

Konverterer de spesifiserte kolonnene til int og float i hoved-DataFrame-en
Dette sikrer at datatypene er riktig formatert for videre analyser og modelltrening

## Fyller inn manglende data med anbefalt imputeringsverdier

In [87]:
# Verdier hentet fra https://archive.ics.uci.edu/dataset/880/support2
fill_values = {
    'serumalbumin': 3.5,
    'lungefunksjon': 333.3,
    'bilirubin': 1.01,
    'kreatinin': 1.01,
    'blodurea_nitrogen': 6.51,
    'hvite_blodlegemer': 9,
    'urinmengde': 2502
}

# Fill missing values in the DataFrame using the dictionary-based approach
pasient_info_df.fillna(fill_values, inplace=True)
pasient_info_df_sample.fillna(fill_values, inplace=True)

Definerer standardverdier for utvalgte variabler der det mangler data.
Disse standardverdiene er typiske median- eller gjennomsnittsverdier som brukes for å imitere realistiske målinger
og bidra til konsistent fylling av manglende data.

## Deler inn i trenings-, validerings- og testdata

In [88]:
df_train, df_valtest = train_test_split(pasient_info_df, test_size=0.3, random_state=0)
df_val, df_test = train_test_split(df_valtest, test_size=0.5, random_state=0)

## Laster opp renset data deling

In [89]:
df_train.to_csv('train_df.csv', index=False)
df_val.to_csv('validation_df.csv', index=False)
df_test.to_csv('test_df.csv', index=False)
pasient_info_df_sample.to_csv('sample_test.csv', index=False)